<a href="https://colab.research.google.com/github/lucaskydelima/Optimization-with-Python-Pyomo/blob/main/S3Example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pyomo
! apt-get install -y -qq glpk-utils

In [5]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [13]:
# Defining the model
model = pyo.ConcreteModel()

# Set 
model.i = pyo.Set(initialize=('compact', 'midsize', 'large'))

# Params
model.phi = pyo.Param(model.i, initialize={'compact':2000, 'midsize':3000, 'large':4000})

model.mu = pyo.Param(model.i, initialize={'compact':1000, 'midsize':1000, 'large':1000})

model.alpha = pyo.Param(model.i, initialize={'compact':{'steel': 1.5, 'labor': 30},
                                             'midsize':{'steel':3, 'labor':25},
                                             'large':{'steel':5, 'labor':40 }})
model.sigma = pyo.Param(initialize=6000)

model.gamma = pyo.Param(initialize=60000)

# Defining big M parameter to be used as upper bound for variable x
model.M = pyo.Param(model.i, initialize={'compact':2000, 'midsize':2000, 'large':1200})

# Function that returns bounds on variable x
def bounds_for_x(model, l):
  return (0, model.M[l])
# Here, l is just a placeholder for the set i

# Decision Variables
model.x = pyo.Var(model.i, domain=pyo.NonNegativeIntegers, bounds=bounds_for_x)
# Note: bounds for x: lower bound = 0, upper bound depends on big M value - define M as parameter first!

model.z = pyo.Var(model.i, domain=pyo.Binary)

# Objective Function
def Objective_rule(model):
  return sum(model.x[i]*model.phi[i] for i in model.i)

model.obj = pyo.Objective(rule=Objective_rule, sense=pyo.maximize)

# Constraints
def Constraint_1(model):
  return sum(model.x[i]*model.alpha[i]['steel'] for i in model.i) <= model.sigma

model.const1 = pyo.Constraint(rule=Constraint_1, doc = 'Steel constraint')

def Constraint_2(model):
  return sum(model.x[i]*model.alpha[i]['labor'] for i in model.i) <= model.gamma

model.const2 = pyo.Constraint(rule=Constraint_2, doc = 'Labor constraint')

def Constraint_3(model, i):
  return model.x[i] <= model.z[i]*model.M[i]

model.const3 = pyo.Constraint(model.i, rule=Constraint_3, doc = 'Part 1 for the either-or constraint equivalent for each car')

def Constraint_4(model, i):
  return model.mu[i] - model.x[i] <= model.M[i]*(1-model.z[i])

model.const4 = pyo.Constraint(model.i, rule=Constraint_4, doc = 'Part 2 for the either-or constraint equivalent for each car')


# Solve
results = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model)

results.write()
print('\n RESULTS \n')
print('Objective Function =', model.obj())
for i in model.i:
  print('Number of cars of type', i, 'will be produced is =>', model.x[i]())

# Model listing - useful for debugging purposes!
# model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 6000000.0
  Upper bound: 6000000.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 7
  Number of nonzeros: 19
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.010562658309936523
# ----------------------------------------------------------
#   Solution Information
# ---------------------------